# Estudio de los datos de Ventas de una Ferretería

Autor: Diana Chacón Ocariz

## Contexto:

Se trata de una pequeña ferretería que maneja un poco más de 3.000 productos distintos. Poseen un software de gestión genérico que les provee una gran cantidad de reportes, básicamente tablas con números, díficiles de analizar (un reporte puede constar de varias decenas de páginas).


## Objetivos del negocio:

**Tener mayor visibilidad sobre las ventas para poder mejorar el proceso de compras y la toma de decisiones en general:** 

    - Poder analizar objetivamente las ventas
    - Determinar los productos que podrían entrar en rotura de stock al final del período
    - Identificar los productos menos vendidos
    - Identificar patrones en el comportamiento de las ventas para poder hacer predicciones de ventas
    

## Objetivos académicos:

    - Estudiar un caso real, con datos reales y cuyo resultado pueda ayudar a alguien a resolver un problema 
    - Demostrar que la ciencia de datos también puede ayudar a las PYMES
    - Conocer y practicar el uso de herramientas de ciencia de datos
    
## Fuentes de datos:

Los datos provienen de reportes sacados del software de gestión de la empresa. Se trata de archivos .xls que contienen sólo los datos de reportes sobre ventas por producto (2021 y 2022) y el stock al final del período. 

# Notebook 2: EDA: Análisis y Visualización

Recuperamos los datos (ya limpios) de los archivos **parquet** para comenzar un análisis más profundo de los datos, responder preguntas del negocio, verificar si hay patrones en las ventas y preparar los datos para utilizarlos en modelos predictivos.

Utilizaremos varias librerías de visualización con el fin de compararlas: **Matplotlib, Seaborn y Altair**

In [1]:
# Librerías utilizadas

import os
import glob
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

import altair as alt

In [2]:
BASE_DIR = Path.cwd()
BASE_DIR

PosixPath('/home/diana/Documentos/Ciencia de Datos/Proyecto Ventas')

In [3]:
%%time 
df_ventas = pd.read_parquet(f"{BASE_DIR / 'datos/out/ventas.parquet'}", engine='fastparquet')
df_ventas

CPU times: user 3.7 s, sys: 80.2 ms, total: 3.78 s
Wall time: 3.85 s


,num,fecha_comp,cliente,vendedor,monto,tipo,cod,producto,cantidad,fecha,tasa_dolar,monto_dolar
0,2020-0000000001-ne,2020-01-07 11:04:00,18018450.0,13,170601.20,ne,,,0.0,2020-01-07,67581.00,2.524396
1,2020-0000000022-fa,2020-01-07 08:57:00,10747595.0,11,1377605.59,fa,,,0.0,2020-01-07,67581.00,20.384510
2,2020-0000000023-fa,2020-01-07 07:52:00,14281493.0,7,623407.20,fa,,,0.0,2020-01-07,67581.00,9.224593
3,2020-0000000024-fa,2020-01-07 09:25:00,19339734.0,7,8520323.59,fa,,,0.0,2020-01-07,67581.00,126.075725
4,2020-0000000025-fa,2020-01-07 09:20:00,16788717.0,13,490901.59,fa,,,0.0,2020-01-07,67581.00,7.263899
...,...,...,...,...,...,...,...,...,...,...,...,...
40916,2022-0000006497-fa,2022-02-05 11:55:00,13763788.0,13,1.19,fa,00809,CONFITERIA TORONTO SAVOY,1.0,2022-02-05,4.60,0.258696
40917,2022-0000006497-fa,2022-02-05 11:55:00,13763788.0,13,1.66,fa,01398,CONFITERIA PASTILLAS CHAO SANDIA/ CEREZA,1.0,2022-02-05,4.60,0.360870
40918,2022-0000006497-fa,2022-02-05 11:55:00,13763788.0,13,6.90,fa,01404,CONFITERIA OREO TUBITO,1.0,2022-02-05,4.60,1.500000
40919,2022-0000006498-fa,2022-02-05 11:58:00,10743720.0,1,69.75,fa,04072,CERRADURA MANILLA RECTA ALUMINIO TOC,1.0,2022-02-05,4.60,15.163043


# Análisis de las Ventas

# Variación de las Ventas en el tiempo

In [4]:
df_ventas_fecha = df_ventas.loc[:,['fecha', 'num', 'cantidad', 'monto_dolar']]
df_ventas_fecha.set_index('fecha', inplace=True)
df_ventas_fecha

,num,cantidad,monto_dolar
fecha,,,
2020-01-07,2020-0000000001-ne,0.0,2.524396
2020-01-07,2020-0000000022-fa,0.0,20.384510
2020-01-07,2020-0000000023-fa,0.0,9.224593
2020-01-07,2020-0000000024-fa,0.0,126.075725
2020-01-07,2020-0000000025-fa,0.0,7.263899
...,...,...,...
2022-02-05,2022-0000006497-fa,1.0,0.258696
2022-02-05,2022-0000006497-fa,1.0,0.360870
2022-02-05,2022-0000006497-fa,1.0,1.500000


### Conclusiones:

    1) En promedio de venden 409 productos cada día
    2) El promedio de ventas diario es de 673 dólares
    3) 41 es el promedio de facturas que se hacen diariamente
    4) Se nota un incremento de las ventas a partir del 2do semestre del 2021
    5) Entre marzo y junio de 2021 se resgistraron la menor cantidad de ventas del período estudiado

## Evolución de las Ventas Mes a Mes

In [5]:
# Utilizamos nunique para contabilizar sólo una vez cada referencia de factura
df_ventas_ag = df_ventas.pivot_table(index='fecha', values=['num', 'cantidad', 'monto_dolar'], 
                                        aggfunc={'num':'nunique', 'cantidad':sum, 'monto_dolar':sum })
df_ventas_ag.reset_index(inplace=True)
df_ventas_ag

,fecha,cantidad,monto_dolar,num
0,2020-01-06,0.00,104.610817,21
1,2020-01-07,0.00,739.473869,59
2,2020-01-08,0.00,485.385935,64
3,2020-01-09,0.00,515.495960,65
4,2020-01-10,0.00,823.755195,66
...,...,...,...,...
597,2022-02-10,31.00,816.187364,6
598,2022-02-11,111.00,252.264642,8
599,2022-02-12,139.90,176.760259,22
600,2022-02-14,126.10,471.782609,15


In [6]:
df_ventas_ag['mes_anio'] = df_ventas_ag.fecha.dt.strftime('%m-%Y')
df_ventas_ag['dia_semana'] = df_ventas_ag.fecha.dt.weekday
df_ventas_ag['dia_mes'] = df_ventas_ag.fecha.dt.day
df_ventas_ag

,fecha,cantidad,monto_dolar,num,mes_anio,dia_semana,dia_mes
0,2020-01-06,0.00,104.610817,21,01-2020,0,6
1,2020-01-07,0.00,739.473869,59,01-2020,1,7
2,2020-01-08,0.00,485.385935,64,01-2020,2,8
3,2020-01-09,0.00,515.495960,65,01-2020,3,9
4,2020-01-10,0.00,823.755195,66,01-2020,4,10
...,...,...,...,...,...,...,...
597,2022-02-10,31.00,816.187364,6,02-2022,3,10
598,2022-02-11,111.00,252.264642,8,02-2022,4,11
599,2022-02-12,139.90,176.760259,22,02-2022,5,12
600,2022-02-14,126.10,471.782609,15,02-2022,0,14


In [7]:
lineas = alt.Chart(df_ventas_ag).mark_line().encode(
    x='fecha:T',
    y='monto_dolar:Q',
    color=alt.Color('yearmonth(fecha):O', scale=alt.Scale(scheme='goldgreen')),
    tooltip=[
        alt.Tooltip('fecha:T', title='Fecha'),
        alt.Tooltip('monto_dolar:Q', title='Ventas en $')
    ]).properties(width=800, height=200)

lineas

alt.Chart(...)

In [8]:
lineas = alt.Chart(df_ventas_ag).mark_line().encode(
    x='fecha:T',
    y='cantidad:Q',
    color=alt.Color('yearmonth(fecha):O', scale=alt.Scale(scheme='purpleblue')),
    tooltip=[
        alt.Tooltip('fecha:T', title='Fecha'),
        alt.Tooltip('cantidad:Q', title='Volumen de Venta')
    ]).properties(width=800, height=200)

lineas

alt.Chart(...)

In [9]:
lineas = alt.Chart(df_ventas_ag).mark_line().encode(
    x='fecha:T',
    y='num:Q',
    color=alt.Color('yearmonth(fecha):O', scale=alt.Scale(scheme='goldorange')),
    tooltip=[
        alt.Tooltip('fecha:T', title='Fecha'),
        alt.Tooltip('num:Q', title='N° Facturas')
    ]).properties(width=800, height=200)

lineas

alt.Chart(...)

In [10]:
df_ventas_dia_semana = df_ventas_ag.pivot_table(index='mes_anio',
                                            columns='dia_semana', aggfunc={'monto_dolar': sum},
                                            fill_value=0)
df_ventas_dia_semana 

monto_dolar                                                      \
dia_semana            0            1            2            3            4   
mes_anio                                                                      
01-2020     1531.657232  2730.974894  1557.202647  2207.919301  2123.907868   
01-2021     2950.556683  4039.964339  2166.060704  2864.239439  2884.332234   
01-2022     5993.008193  6345.862615  4211.000312  3849.191008  4383.430025   
02-2020     1563.447720  2453.311830  2397.020698  1701.697084  4424.442111   
02-2021     1559.886233  3585.569561  3141.892730  3451.844984  3229.538314   
02-2022      549.006036   496.764451   473.216378   894.183072   644.988555   
03-2020     3220.645464  2652.192226  2903.512438  1892.565451  1937.839969   
03-2021     3298.064658  2713.807813  2568.880136  2812.007238  1635.795558   
04-2020     1027.444224  1645.236462  2942.525720  1801.916525  3949.732841   
04-2021     2468.683163  3202.946005  2310.648747  2619.331624  2941.064728   
05-2020     7854.959273  2528.419154  4570.174850  5999.999829  3050.882230   
05-2021     2883.251353  1626.363609  5894.606022  1447.094554  1673.973381   
06-2020     3833.685642  3105.616794  2177.094470  1602.669703  2273.071456   
06-2021     3612.375030  3481.414043  2961.057943  1507.987450  2389.329167   
07-2020     2431.498008  3196.185788  3088.079515  3774.515277  4449.881610   
07-2021     2942.644847  3828.785077  3079.710761  3582.390103  4408.330709   
08-2020     4043.862722  2774.204571  2655.105064  1249.315214  3383.577080   
08-2021     3938.888904  3817.415182  3628.801368  1926.945845  1883.957056   
09-2020     2785.093720  2772.080027  2630.947624  1229.478122  1975.287385   
09-2021     3341.969818  4281.933775  3764.275245  3488.371117  2681.302988   
10-2020     2679.719593  1720.262033  1439.657625  3107.864567  9712.804210   
10-2021     5572.556618  3122.798839  4719.426861  2731.650867  2721.634103   
11-2020     2747.359079  2988.822244  1737.501192  2559.723768  1395.196235   
11-2021     4379.532099  4331.640255  3098.016685  2598.875623  3435.663573   
12-2020     2234.683248  3309.964875  3937.292916  2323.513895  1245.458278   
12-2021     3477.849793  2700.962950  4476.155053  3176.981472  4952.909467   

                                      
dia_semana            5            6  
mes_anio                              
01-2020       82.358816   333.857234  
01-2021      996.294727     0.000000  
01-2022     4456.175386     0.000000  
02-2020        0.000000   697.923364  
02-2021     1140.054082     0.000000  
02-2022      194.042868     0.000000  
03-2020        0.000000  1693.889650  
03-2021      480.829274     0.000000  
04-2020        0.000000     0.000000  
04-2021      437.554386     0.000000  
05-2020        0.000000     0.000000  
05-2021      801.505309     0.000000  
06-2020        0.000000     0.000000  
06-2021      920.295099     0.000000  
07-2020      154.672530     0.000000  
07-2021     1855.761776     0.000000  
08-2020        7.996093   276.899321  
08-2021     1078.645626     0.000000  
09-2020      122.196078     0.000000  
09-2021     1117.952683     0.000000  
10-2020        0.000000     0.000000  
10-2021     1649.615102     0.000000  
11-2020      290.431152   131.577089  
11-2021     1548.805023     0.000000  
12-2020      777.747714     0.000000  
12-2021      891.800771     0.000000

In [11]:
mapa = alt.Chart(df_ventas_ag).mark_rect().encode(
                x=alt.X('day(fecha):T', title='Días de la Semana', axis = alt.Axis(labelAngle=0, labelFontSize=14)),
                y=alt.Y('yearmonth(fecha):T', title = 'Ventas en $', scale=alt.Scale(zero=False), 
                  axis = alt.Axis(grid=True, titleAnchor='middle', titleAngle = 270, labelFontSize=10)),
                color=alt.Color('sum(monto_dolar):Q', scale=alt.Scale(scheme='goldgreen'), title='Ventas en $'),
                tooltip=[
                    alt.Tooltip('day(fecha):T', title='Día'),
                    alt.Tooltip('yearmonth(fecha):T', title='Mes y Año'),
                    alt.Tooltip('sum(monto_dolar):Q', title='Ventas en $')]
                ).properties(title='Ventas en $ por Día de la Semana',
                             width=300, 
                             height=300
                ).configure_title(
                    fontSize = 16,
                    anchor = 'middle'
                ).interactive()

mapa

alt.Chart(...)

In [12]:
barras = alt.Chart(df_ventas_ag).mark_bar().encode(
                x=alt.X('day(fecha):T', title='Días de la Semana', axis = alt.Axis(labelAngle=0, labelFontSize=14)),
                y=alt.Y('sum(monto_dolar):Q', title = 'Ventas en $', scale=alt.Scale(zero=False), 
                  axis = alt.Axis(grid=True, titleAnchor='middle', titleAngle = 270, labelFontSize=10)),
                color=alt.Color(
                    'sum(monto_dolar):Q', scale=alt.Scale(scheme='goldgreen'), title='Ventas en $'),
                tooltip=[
                    alt.Tooltip('day(fecha):T', title='Día'),
                    alt.Tooltip('sum(monto_dolar):Q', title='Ventas en $')],
                ).properties(title='Ventas en $ por Día de la Semana',
                             width=600, 
                             height=300
                ).configure_title(
                    fontSize = 16,
                    anchor = 'middle'
                ).interactive()
barras

alt.Chart(...)

In [13]:
df_ventas_dia_mes = df_ventas_ag.pivot_table(index='mes_anio',
                                            columns='dia_mes', aggfunc={'monto_dolar': sum},
                                            fill_value=0)
df_ventas_dia_mes

monto_dolar                                                      \
dia_mes            1            2            3            4            5    
mes_anio                                                                    
01-2020      0.000000     0.000000     0.000000     0.000000     0.000000   
01-2021      0.000000     0.000000     0.000000     0.000000  1025.174371   
01-2022      0.000000     0.000000  1628.923241   773.260504   854.500000   
02-2020      0.000000   228.427405   591.898212   840.911939   434.875294   
02-2021    597.742987   928.112308   448.788414   676.658349  1448.390864   
02-2022     24.791045   109.068230    77.995708   392.723913    17.282609   
03-2020    102.471183  1569.100584   740.156960  1213.586690   581.002611   
03-2021   1282.797023   327.117997   521.856075  1258.638821   606.867712   
04-2020    429.947573   343.727378  1865.040038     0.000000     0.000000   
04-2021      0.000000     0.000000     0.000000     0.000000   310.550924   
05-2020      0.000000     0.000000     0.000000   559.629421   708.440722   
05-2021      0.000000     0.000000   179.121985   229.004501   303.164561   
06-2020    984.311972  1332.024144   696.307495   323.136042   935.775980   
06-2021   1610.459773   645.720572   618.434072   660.443987   176.276599   
07-2020    460.703161   830.441806   554.543131     0.000000     0.000000   
07-2021    449.820795   670.552823   205.902734     0.000000   665.648431   
08-2020      0.000000     0.000000  1197.170954   556.828608     0.000000   
08-2021      0.000000  2076.398369   532.983106   908.458977     0.000000   
09-2020    696.059023   780.140801   336.050436   478.429830     0.000000   
09-2021    351.150160   407.752358   306.196339   286.529001     0.000000   
10-2020   1637.070887  1010.448421     0.000000     0.000000   938.159789   
10-2021    489.654244   302.497817     0.000000  2547.654501   889.681818   
11-2020      0.000000   421.493914  1279.945126   393.706990   992.169576   
11-2021    511.719101   795.961538   634.821670   915.937639   444.185268   
12-2020    834.225726   450.752098   456.728855   577.661259   289.318192   
12-2021   1804.950207   437.373695  1142.676471   262.088050     0.000000   

                                                                           \
dia_mes            6            7            8            9            10   
mes_anio                                                                    
01-2020    104.610817   739.473869   485.385935   515.495960   823.755195   
01-2021    338.556452   559.099981   406.877705   113.560197     0.000000   
01-2022   1579.416327  1191.730392   698.201474     0.000000  1482.000000   
02-2020    582.694724   568.515668     0.000000   288.046326   395.750196   
02-2021    285.706649     0.000000   528.333392  1052.128996   945.011517   
02-2022      0.000000    77.223427   224.206522   364.148148   816.187364   
03-2020    646.227529     0.000000  1359.571616   940.063855   918.230849   
03-2021    138.317334     0.000000   566.644612   696.262891   616.013044   
04-2020      0.000000     0.000000     0.000000     0.000000     0.000000   
04-2021    577.616637   288.305906   468.976941   244.816049   144.322024   
05-2020    878.469839   492.796971   864.247780     0.000000     0.000000   
05-2021    229.910797   125.169426   138.451233     0.000000   337.713740   
06-2020      0.000000     0.000000   462.697836   422.152875   749.672755   
06-2021      0.000000   732.745323   457.801844   438.424317   296.985811   
07-2020    357.390229  1427.228788   680.721010   798.203946   871.979596   
07-2021   1407.765877  1281.355950  1413.361915  1104.303772   448.666178   
08-2020      0.000000  1479.949875     0.000000   276.899321  1156.194226   
08-2021      0.000000   536.191117     0.000000   720.760998   567.037550   
09-2020      0.000000   543.376057   648.269859   256.875271   204.540011   
09-2021    493.153545  2038.773379  1533.587377   658.470572   798.573567   
10-20

In [14]:
mapa = alt.Chart(df_ventas_ag).mark_rect().encode(
                x=alt.X('dia_mes:O', title='Días del mes', axis = alt.Axis(labelAngle=0, labelFontSize=14)),
                y=alt.Y('yearmonth(fecha):T', title = 'Mes y Año', scale=alt.Scale(zero=False), 
                  axis = alt.Axis(grid=True, titleAnchor='middle', titleAngle = 270, labelFontSize=10)),
                color=alt.Color(
                    'sum(monto_dolar):Q', scale=alt.Scale(scheme='goldgreen'), title='Ventas en $'),
                tooltip=[
                    alt.Tooltip('dia_mes:O', title='Día'),
                    alt.Tooltip('yearmonth(fecha):T', title='Mes y Año'),
                    alt.Tooltip('sum(monto_dolar):Q', format=',.2f', title='Ventas en $'),
                    alt.Tooltip('cantidad:Q', format=',.2f', title='Volumen de Ventas'),
                    alt.Tooltip('num:Q', title='N° Facturas')]
                ).properties(title='Ventas en $ por Día del Mes',
                             width=800,  
                             height=300
                ).configure_title(
                    fontSize = 16,
                    anchor = 'middle'
                ).interactive()

mapa

alt.Chart(...)

In [15]:
barras = alt.Chart(df_ventas_ag).mark_bar().encode(
                x=alt.X('dia_mes:O', title='Días del Mes', axis = alt.Axis(labelAngle=0, labelFontSize=14)),
                y=alt.Y('sum(monto_dolar):Q', title = 'Ventas en $', scale=alt.Scale(zero=False), 
                  axis = alt.Axis(grid=True, titleAnchor='middle', titleAngle = 270, labelFontSize=10)),
                color=alt.Color(
                    'sum(monto_dolar):Q', scale=alt.Scale(scheme='goldgreen'), title='Ventas en $'),
                tooltip=[
                    alt.Tooltip('dia_mes:O', title='Día'),
                    alt.Tooltip('sum(monto_dolar):Q', title='Ventas en $')],
                ).properties(title='Ventas en $ por Día del Mes',
                             width=800, 
                             height=300
                ).configure_title(
                    fontSize = 16,
                    anchor = 'middle'
                ).interactive()
barras

alt.Chart(...)

Los lunes y martes son los días en los que más se vende en $. El viernes en Volumen

Los jueves son los días que menos se vende, que sea en volumen o en $

### Análisis por mes

In [16]:
lineas = alt.Chart(df_ventas_ag).mark_bar().encode(
    x='dia_mes:O',
    y='monto_dolar:Q',
    color=alt.Color('yearmonth(fecha):T', scale=alt.Scale(scheme='goldgreen')),
    column=alt.Column('yearmonth(fecha):T', title='yearmonth(fecha)'),
    tooltip=[
        alt.Tooltip('fecha:T', title='Fecha'),
        alt.Tooltip('monto_dolar:Q', title='Ventas en $'),
    ]).properties(width=800, height=200)

lineas

alt.Chart(...)